In [1]:
import three_desc_model as exponential_new
import numpy as np
import pandas as pd
from cutoffs import Polynomial
#import tensorflow as tf
import matplotlib.pyplot as plt
#import keras
#from tensorflow.keras import layers
#from tensorflow.keras.layers.experimental import preprocessing
from scipy.optimize import minimize
turbs = ["Siemens"] *2
turbs_3 = ["Siemens"] *3
nl=exponential_new.NeighborlistCalculator(cutoff=4001,cone_grad=0.12582561117875557, cone_offset=72.24947126849844)
Gs = {"Siemens": [{"type":"G2", "turbine":"Siemens","eta":1, "offset": 1.},
                  {"type":"G4", "elements":"Siemens","eta":1, "gamma":4., "zeta": 1.0},
                  {"type":"G6", "elements":"Siemens","eta":1, "gamma": 0.5, "zeta": 1.0}
                 ]}
finpr=exponential_new.FingerprintCalculator(cutoff=4001,Gs=Gs,Rct=3000,delta_R=100,cone_grad=0.12582561117875557, cone_offset=72.24947126849844)

symbol="Siemens"
dataset_pos=np.empty(shape=(0, 2))
dataset_fp=np.empty(shape=(0, 3))
dataset_ws=np.empty(shape=(0, 1))

In [ ]:
def build_and_compile_model(norm):
    model = keras.Sequential([
        norm,
        layers.Dense(64, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(16, activation='relu'),
        layers.Dense(1)
    ])
    model.compile(loss='mean_absolute_error', optimizer=tf.keras.optimizers.Adam(0.001))
    return model

In [2]:
dataset = pd.read_csv('all_dataset.csv', index_col=0)
dataset

,X_coord,Y_coord,Ref_wind,Num_tot_turb,ID,Turb_num
0,0.000000,0.000000,7.479762,2.0,2_turbine_240_degrees_279.0_meters,0.0
1,241.621088,139.500000,7.576231,2.0,2_turbine_240_degrees_279.0_meters,1.0
2,0.000000,0.000000,7.473602,2.0,2_turbine_241_degrees_279.0_meters,0.0
3,244.018898,135.261884,7.567527,2.0,2_turbine_241_degrees_279.0_meters,1.0
4,0.000000,0.000000,7.467848,2.0,2_turbine_242_degrees_279.0_meters,0.0
...,...,...,...,...,...,...
1779,3164.356744,-110.000000,6.978195,3.0,3_turbine_358th_simulation,1.0
1780,499.388946,-80.000000,6.763982,3.0,3_turbine_358th_simulation,2.0
1781,0.000000,0.000000,7.306684,3.0,3_turbine_359th_simulation,0.0
1782,2630.260284,143.000000,7.232828,3.0,3_turbine_359th_simulation,1.0


In [3]:
k=0
count=0
dataset["IDnum"]=pd.Series()
for i in range(len(dataset)):
    dataset.at[i,"IDnum"]=k
    #print(i,k,dataset["Num_tot_turb"].iloc[i])
    count = count +1
    if (count==dataset["Num_tot_turb"].iloc[i]):
        k=k+1
        count=0
dataset["Num_tot_turb"]=dataset["Num_tot_turb"].astype(int)
dataset["Turb_num"]=dataset["Turb_num"].astype(int)
dataset["IDnum"]=dataset["IDnum"].astype(int)
dataset

/var/folders/gr/z6l2h7kd4136nf0xqdtls40w0000gn/T/ipykernel_19661/2135225642.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  dataset["IDnum"]=pd.Series()


,X_coord,Y_coord,Ref_wind,Num_tot_turb,ID,Turb_num,IDnum
0,0.000000,0.000000,7.479762,2,2_turbine_240_degrees_279.0_meters,0,0
1,241.621088,139.500000,7.576231,2,2_turbine_240_degrees_279.0_meters,1,0
2,0.000000,0.000000,7.473602,2,2_turbine_241_degrees_279.0_meters,0,1
3,244.018898,135.261884,7.567527,2,2_turbine_241_degrees_279.0_meters,1,1
4,0.000000,0.000000,7.467848,2,2_turbine_242_degrees_279.0_meters,0,2
...,...,...,...,...,...,...,...
1779,3164.356744,-110.000000,6.978195,3,3_turbine_358th_simulation,1,710
1780,499.388946,-80.000000,6.763982,3,3_turbine_358th_simulation,2,710
1781,0.000000,0.000000,7.306684,3,3_turbine_359th_simulation,0,711
1782,2630.260284,143.000000,7.232828,3,3_turbine_359th_simulation,1,711


In [4]:
numsims=dataset["IDnum"].iloc[-1]+1
print(numsims)
turb = [symbol]

712


In [7]:
count=0
for i in range(numsims):
#    if dataset.loc[dataset['Number of Turbine in the Simulation'] == 2].reset_index(drop=True)
    numturb=dataset["Num_tot_turb"].iloc[count]
    position = np.empty((numturb,2))
    ws = np.empty((numturb,1))
    fp = np.empty((numturb,3))
    for k in range(numturb):
        position[k,0]=dataset.at[count,"X_coord"]
        position[k,1]=dataset.at[count,"Y_coord"]
        ws[k,0]=dataset.at[count,"Ref_wind"]
        count = count+ 1
    neigh=nl.calculate(turb*numturb,position)
    fp=finpr.calculate(turb*numturb,position,neigh,symbol)
    fp=np.array(fp)
#    print(fp)
    dataset_pos=np.append(dataset_pos,position,axis=0)
    dataset_fp=np.append(dataset_fp,fp,axis=0)
    dataset_ws=np.append(dataset_ws,ws,axis=0)
dataset_full=np.concatenate((dataset_fp, dataset_ws),axis=1)
dataset_full = pd.DataFrame(dataset_full, columns = ['Fingerprint(G2)','Fingerprint(G4)','Fingerprint(G6)','Ref_Wind_Speed'])
dataset_full

,Fingerprint(G2),Fingerprint(G4),Fingerprint(G6),Ref_Wind_Speed
0,1.000000,1.000000,1.000000,7.479762
1,1.000000,1.000000,1.000000,7.576231
2,1.000000,1.000000,1.000000,7.576231
3,1.000000,1.000000,1.000000,7.576231
4,1.000000,1.000000,1.000000,7.576231
...,...,...,...,...
4987,0.266500,0.207342,0.473401,6.978195
4988,0.233438,0.546630,1.000000,6.763982
4989,1.000000,1.000000,1.000000,7.306684
4990,0.209439,0.269102,0.394721,7.232828


In [ ]:
def fitness(x):
    Gs = {"Siemens": [{"type":"G2", "turbine":"Siemens","eta":x[0], "offset": 1.},
                  {"type":"G4", "elements":"Siemens","eta":x[1], "gamma":4., "zeta": 1.0},
                  {"type":"G6", "elements":"Siemens","eta":x[2], "gamma": 0.5, "zeta": 1.0}
                 ]}
    turbs_3 = ["Siemens"] *3
    nl=exponential_new.NeighborlistCalculator(cutoff=4001,cone_grad=0.12582561117875557, cone_offset=72.24947126849844)


    symbol="Siemens"

    
    dataset = pd.DataFrame(dataset, columns = ['Fingerprint(G2)','Fingerprint(G4)','Fingerprint(G6)','Ref_Wind_Speed'])
    train_dataset = dataset.sample(frac=0.8, random_state=0)
    test_dataset = dataset.drop(train_dataset.index)
    train_features = train_dataset.copy()
    test_features = test_dataset.copy()
    all_features = dataset.copy()
    train_labels = train_features.pop('Ref_Wind_Speed')
    test_labels = test_features.pop('Ref_Wind_Speed')
    all_labels = all_features.pop('Ref_Wind_Speed')
    normalizer = preprocessing.Normalization(axis=-1)
    normalizer.adapt(np.array(train_features))
    dnn_model = build_and_compile_model(normalizer)
    history = dnn_model.fit(
        train_features, train_labels,
        validation_split=0.2,
        verbose=0, epochs=120)
    all_predictions = dnn_model.predict(all_features).flatten()
    error_all=np.abs(all_predictions-all_labels)
    return np.max(error_all)

In [ ]:
x0 = [5.3, 5., 2.]
bounds= ((0,10),(0,10),(0,10))
res = minimize(fitness, x0, bounds= bounds, options={'disp':True})

In [ ]:
for i,row in dataset.iterrows():
    if row['Num_tot_turb'] == 3:
        position = np.array([[dataset.at[i,"X_coord"],dataset.at[i,"Y_coord"]],[dataset.at[i,"X_coord"],id_dataset.at[1,"Y_coord"]],[id_dataset.at[2,"X_coord"],id_dataset.at[2,"Y_coord"]]])